In [1]:
import pandas as pd
import numpy as np
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
import re
df=pd.read_excel('Maintenance_data_w_VIN_full.xlsx')

In [2]:
df.shape

(991, 18)

In [3]:
df.drop_duplicates(subset='maintenance_id', keep='first', inplace=True)
df.shape

(542, 18)

In [4]:


# Function to check if a description is empty or a time string
def is_invalid_description(desc):
    # Convert desc to string in case it's not (handles NaN and None)
    desc = str(desc)
    if not desc or desc == 'nan':
        return True
    # Regex to check if the description is just a time string
    time_pattern = r'^\d{1,2}/\d{1,2}/\d{2,4} \d{1,2}:\d{2} (AM|PM)\s*$'

    return bool(re.match(time_pattern, desc))


# Filter out rows with invalid descriptions
df = df[~df['description'].apply(is_invalid_description)]

df



,Unnamed: 0,VIN_id,maintenance_id,date_created,date_updated,estimated_delivery_date,requested_delivery_date,assigned_vendor_id,inspection_type_id,date_completed,vendor_contacted_date,vendor_email,hours,available_pickup_date,in_house,status,work_order,description
2,2.0,NaN,71751629.0,04-10-2018,NaN,NaN,NaN,PROGRESSIVE DIESEL LTD,NaN,NaN,04-10-2018,NaN,NaN,NaN,False,Delivered,71751629.0,04/10/2018 1:54 PM 04/10/2018 01:54 PM : Auto_...
11,11.0,NaN,13306496.0,02-19-2007,NaN,NaN,NaN,HARBOURS END MARINE,NaN,NaN,02-19-2007,NaN,NaN,NaN,False,Delivered,13306496.0,02/19/2007 3:19 PM 02/19/2007 03:19 PM : 200HR...
13,13.0,NaN,36646116.0,07-16-2012,NaN,NaN,NaN,PROGRESSIVE DIESEL LTD,NaN,NaN,07-16-2012,NaN,NaN,NaN,False,Delivered,36646116.0,07/16/2012 7:32 PM 07/16/2012 07:32 PM : WO 41...
15,15.0,NaN,89669554.0,06-14-2021,NaN,NaN,NaN,PROGRESSIVE DIESEL LTD,NaN,NaN,06-14-2021,NaN,949.0,NaN,False,Delivered,89669554.0,06/14/2021 9:10 PM 06/14/2021 09:10 PM : Auto_...
16,16.0,NaN,87502005.0,01-19-2021,NaN,NaN,NaN,PROGRESSIVE DIESEL LTD,NaN,NaN,01-19-2021,NaN,906.0,NaN,False,Delivered,87502005.0,01/19/2021 6:27 PM 01/19/2021 06:27 PM : Auto_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,983.0,NaN,85055923.0,08-24-2020,NaN,NaN,NaN,R G'S AUTO SERVICES LTD,NaN,NaN,08-24-2020,NaN,NaN,NaN,False,Delivered,85055923.0,07/07/2020 3:00 AM 08/24/2020 01:30 PM : WO# 5...
984,984.0,NaN,97920921.0,01-05-2023,NaN,NaN,NaN,RG'S AUTO MARINE LTD,NaN,NaN,01-05-2023,NaN,757.0,NaN,False,Delivered,97920921.0,01/10/2023 11:56 AM 01/10/2023 11:56 AM : UNAB...
985,985.0,NaN,66381281.0,04-26-2017,NaN,NaN,NaN,BRIDGEVIEW MARINE LTD,NaN,NaN,04-26-2017,NaN,NaN,NaN,False,Delivered,66381281.0,04/26/2017 12:00 PM 04/26/2017 12:25 PM : SERV...
986,986.0,NaN,78752324.0,06-24-2019,NaN,NaN,NaN,BRIDGEVIEW MARINE LTD,NaN,NaN,06-24-2019,NaN,NaN,NaN,False,Delivered,78752324.0,06/25/2019 10:32 AM 06/25/2019 10:32 AM : HUMA...


In [6]:
inspection_keywords = {
    'Annual Inspection': [
        r'ANNUAL INSPECTION', r'INSPECTION DECAL', r'CERTIFICATION', r'COMMERCIAL INSPECTION'
    ],
    'Safety Inspection': [
        r'SAFETY EQUIP INSPT', r'EMERGENCY SERVICE', r'HYDRO STATIC SWITCH', r'RESCUE THROW BAG'
    ],
    'Preventive Maintenance / Service': [
        r'100 HOUR SERVICE', r'300 HR PM SERV', r'FULL SVC', r'WINTERIZE', r'PREVENTIVE MAINT'
    ],
    'Overhaul / Upgrade': [
        r'ENGINE REMOVAL', r'UPGRADE', r'NEW POWER STEERING SYSTEM', r'REPOWERING THEIR FLEET'
    ],
    'Emergency / Urgent Repairs': [
        r'EMERGENCY SERVICE', r'IMMEDIATE', r'URGENT'
    ],
    'Major Inspection': [
        r'REPAIR', r'REPLACE', r'DIAGNOSE', r'PRESSURE/VACUUM TEST', r'PROP SHAFT', r'SERVICE ENGINE', r'CLIENT AUTHORIZATION PROCESS'
    ],
    'Minor Inspection': [
        r'SERVICE', r'CHECK', r'INSPECT', r'MAINTENANCE', r'FULL SERVICE'
    ]
}

# Function to classify inspection based on keywords
def classify_inspection(description):
    # Check for each inspection type keywords
    for inspection_type, keywords in inspection_keywords.items():
        for keyword in keywords:
            if re.search(keyword, description, re.IGNORECASE):
                return inspection_type
    # Default classification if no keywords found
    return 'Unclassified'

In [7]:
df['inspection_type'] = df['description'].apply(classify_inspection)


In [8]:
df['inspection_type'].value_counts()

Major Inspection                    308
Preventive Maintenance / Service    112
Unclassified                         29
Minor Inspection                     25
Safety Inspection                     8
Overhaul / Upgrade                    7
Annual Inspection                     7
Emergency / Urgent Repairs            1
Name: inspection_type, dtype: int64

In [ ]:
#df[(df['inspection_type'] == 'Unclassified')]['description']